In [8]:
import 서울시_지하철_승하차_현황_API as sw_pa
import 서울시유무임승차현황 as sw_py_fr
import pandas as pd
from datetime import datetime, timedelta

In [9]:
def Data_input_all():
    ### 전체 데이터 API에서 가져오기
    sDt = '20150101'
    eDt = datetime.today().strftime('%Y%d%m')
    
    ### 지하철 승하차 인원 데이터 조회 및 저장
    df0=sw_pa.main_api(sDt, eDt)
    df0.to_csv('c:/pydata/seoul_sw_inout.csv', index = False)
    
    ### 지하철 유/무임승하차인원 데이터  조회 및 저장
    df1=sw_py_fr.main_api(sDt[:-2], eDt[:-2])
    df1.to_csv('c:/pydata/seoul_sw_payfree.csv', index = False)
    print('==============ApI 데이터 가져오기 성공=================')
    

## 서울시 지하철 탑승현황
- seoul_sw_inout.csv / c:pydata/seoul_sw_payfree.csv 이용
- 각 파일에서 추가되지 않은 데이터를 API를 이용해 가져와 추가
- 승하차 데이터 / 유임 / 무임 승하차 데이터를 이용해 년도별/월별 데이터 생성
- 역 입력시 해당 역의 승/하차, 유/무임 데이터 확인 및 시각화
- 유임/무임 승하차가 가장 많은 역 확인

### 데이터 읽어오기 및 전처리
- 두 데이터 가져오기
- 오늘 날짜를 기준으로 없는 날짜 추가하기


In [10]:
df0=pd.read_csv('c:/pydata/seoul_sw_inout.csv')
df1=pd.read_csv('c:/pydata/seoul_sw_payfree.csv')

#### 오늘 날짜를 기준으로 없는 데이터 추가하기

- 1. 읽어온 데이터의 마지막 데이터의 날짜/년월을 가져온다
- 2. 마지막 날짜가 오늘날짜/오늘년월 일치/불일치 확인 (일치->종료 , 불일치->API에서 데이터 읽어오기)
- 3. 가져온 데이터 추가 후 pydata 업데이트

In [11]:
if str(df0.iloc[-1, 0])  !=  datetime.today().strftime('%Y%m%d'):
    sDt=datetime.strptime(str(df0.iloc[-1,0]),'%Y%m%d' )+timedelta(1)
    sDt = str(sDt.strftime('%Y%m%d'))
    eDt = str(datetime.today().strftime('%Y%m%d'))
    
    df=sw_pa.main_api(sDt, eDt)
    
    df0 = pd.concat([df0, df], ignore_index = True)
    df0.to_csv('c:/pydata/seoul_sw_inout.csv', index = False)

    print('일자별데이터 업데이트 완료')

진행율: 100%|████████████████████████████████████████████████████████████████████████████| 4/4 [00:17<00:00,  4.28s/it]


일자별데이터 업데이트 완료


In [12]:
df0

,사용일,라인,역명,승차,하차
0,20150101,1호선,서울역,47071,40197
1,20150101,1호선,시청,11016,8943
2,20150101,1호선,종각,22084,17648
3,20150101,1호선,종로3가,17917,15817
4,20150101,1호선,종로5가,9789,9216
...,...,...,...,...,...
1368197,20210705,중앙선,오빈,313,299
1368198,20210705,중앙선,양평,2774,2807
1368199,20210705,중앙선,원덕,356,357
1368200,20210705,중앙선,용문,3142,3093


In [14]:
sDt=datetime.strptime(str(df1.iloc[-1,0])+ "27",'%Y%m%d' )+timedelta(5)

if sDt.strftime('%Y%m') != datetime.today().strftime('%Y%m'):
    sDt= str(sDt.strftime('%Y%m'))
    eDt = str(datetime.today().strftime('%Y%m'))

    df=sw_pa.main_api(sDt, eDt)

    df1 = pd.concat([df1, df], ignore_index = True)
    df1.to_csv('c:/pydata/seoul_sw_payfree.csv', index = False)

print('유무임 데이터 업데이트 완료')

유무임 데이터 업데이트 완료


In [31]:
df0=pd.read_csv('c:/pydata/seoul_sw_inout.csv')
df0=df0.astype({'사용일' : 'str'})
print(df0.info())
df1=pd.read_csv('c:/pydata/seoul_sw_payfree.csv')
df1 = df1.rename(columns = {"사용일" : '사용년월'})
df1 = df1.astype({'사용년월' : 'str'})
print(df1.info())
df1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1368202 entries, 0 to 1368201
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   사용일     1368202 non-null  object
 1   라인      1368202 non-null  object
 2   역명      1368202 non-null  object
 3   승차      1368202 non-null  int64 
 4   하차      1368202 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 52.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40931 entries, 0 to 40930
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   사용년월    40931 non-null  object
 1   라인      40931 non-null  object
 2   역명      40931 non-null  object
 3   유임승차    40931 non-null  int64 
 4   무임승차    40931 non-null  int64 
 5   유임하차    40931 non-null  int64 
 6   무임하차    40931 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 2.2+ MB
None


,사용년월,라인,역명,유임승차,무임승차,유임하차,무임하차
0,201501,4호선,삼각지,138257,178987,32268,36116
1,201501,4호선,신용산,343494,370255,53325,53531
2,201501,4호선,이촌,257256,283880,42962,45709
3,201501,4호선,동작,47701,50075,17183,16453
4,201501,4호선,총신대입구,550301,619606,134289,142082


- 승하차 데이터 / 유무임 승하차 데이터를 이용해 년도별/연별 데이터 생성

In [30]:
df0['사용년월']=df0['사용일'].str[:6]
df2 = df0.groupby(['사용년월','라인','역명']).sum()
df2 = df2.reset_index()
df2

,사용년월,라인,역명,승차,하차
0,201501,1호선,동대문,483789,542499
1,201501,1호선,동묘앞,285550,300658
2,201501,1호선,서울역,2160616,1916768
3,201501,1호선,시청,748569,742981
4,201501,1호선,신설동,512694,495320
...,...,...,...,...,...
45939,202107,중앙선,원덕,1738,1723
45940,202107,중앙선,중랑,24224,23085
45941,202107,중앙선,지평,348,378
45942,202107,중앙선,팔당,4966,4910


In [38]:
# for i in df1.index:
#     print(df2[(df2['사용년월']==df1.loc[i][0])&(df2['라인']==df1.loc[i][1])&(df2['역명']==df1.loc[i][2])])


In [36]:
df0['사용년월']=df0['사용일'].str[:6]

df2 = df0.groupby(['사용년월','라인','역명']).sum()
df3 = df1.groupby(['사용년월','라인','역명']).sum()

In [41]:
#df4=pd.concat([df2,df3], axis = 1, join='outer')#join='outer' ->합집합 , 기본값
df4=pd.concat([df2,df3], axis = 1, join='inner')#join='inner' ->교집합
df4= df4.reset_index()
df4

,사용년월,라인,역명,승차,하차,유임승차,무임승차,유임하차,무임하차
0,201501,1호선,동대문,483789,542499,336981,390926,132944,137814
1,201501,1호선,동묘앞,285550,300658,155556,171171,124838,124116
2,201501,1호선,서울역,2160616,1916768,1890411,1667163,238403,220008
3,201501,1호선,시청,748569,742981,639441,636014,87392,85689
4,201501,1호선,신설동,512694,495320,374981,363032,124078,118929
...,...,...,...,...,...,...,...,...,...
40839,202106,중앙선,원덕,11473,11010,5259,6214,5030,5980
40840,202106,중앙선,중랑,154409,147095,110574,43835,104494,42601
40841,202106,중앙선,지평,1982,2229,872,1110,1152,1077
40842,202106,중앙선,팔당,30925,31638,22609,8316,23186,8452


In [46]:
df4[df4['사용년월']=='202105']['유임승차'].max()

1967326

In [49]:
df4[(df4['사용년월']=='202106')&(df4['무임승차']==df4[df4['사용년월']=='202106']['무임승차'].max())]


,사용년월,라인,역명,승차,하차,유임승차,무임승차,유임하차,무임하차
40243,202106,1호선,종로3가,684833,625095,427162,257671,383856,241239


In [51]:
df4[(df4['사용년월']=='202106')&(df4['무임하차']==df4[df4['사용년월']=='202106']['무임하차'].max())]


,사용년월,라인,역명,승차,하차,유임승차,무임승차,유임하차,무임하차
40241,202106,1호선,제기동,459463,470591,224878,234585,218527,252064


In [54]:
df4[df4['사용년월']=='202106'].sort_values(by="무임승차", ascending=False).head()


,사용년월,라인,역명,승차,하차,유임승차,무임승차,유임하차,무임하차
40243,202106,1호선,종로3가,684833,625095,427162,257671,383856,241239
40244,202106,1호선,종로5가,598069,597269,358583,239486,366760,230509
40245,202106,1호선,청량리(서울시립대입구),553525,555155,317436,236089,314995,240160
40241,202106,1호선,제기동,459463,470591,224878,234585,218527,252064
40319,202106,3호선,연신내,1014329,953790,786054,228275,732219,221571
